In [9]:
import pandas as pd
from dfply import *

import sys
from sshtunnel import SSHTunnelForwarder
import logging
sys.path.append('/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철')

import src.data_extraction.access_database as db   # DB 연결 관련 모듈

## Features: Severity (Charlson, APS III, SAPS II, OASIS)

### 1. Subjectlist 가져오기

In [8]:
# 데이터 저장위치
# data_dir = './data'   # 수정 쌤 디렉토리에서 사용
data_dir = '../../outputs'   # 내 디렉에서 사용
sub_list_dir = data_dir + '/' + 'sub1list.csv'


subjectlist1 = pd.read_csv(sub_list_dir, parse_dates=['admittime', 'dischtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id', 'admittime', 
                                'dischtime', 'gender', 'age', 'height', 
                                'weight')
sub1list = sub1list.drop_duplicates(subset='stay_id')   # remove dup
sub1list

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-09-02 17:05:00,F,75.589414,165.0,99.6
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-13 17:31:00,M,85.274942,NaN,80.4
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-02-06 11:45:00,M,85.059155,168.0,76.0
4,10005817,28661809,31316840,2135-01-03 21:54:00,2135-01-19 18:36:00,M,69.008724,175.0,88.8
...,...,...,...,...,...,...,...,...,...
7226,19995595,21784060,34670930,2126-10-17 21:43:00,2126-11-12 17:35:00,M,68.793734,163.0,94.7
7227,19997367,20617667,35616526,2126-04-20 07:15:00,2126-05-19 14:15:00,F,63.299259,147.0,59.0
7228,19999068,21606769,30143796,2161-08-24 04:10:00,2161-09-02 19:00:00,M,63.643884,170.0,55.8
7229,19999442,26785317,32336619,2148-11-19 10:00:00,2148-12-04 16:25:00,M,43.884161,193.0,107.5


## 2. DB 연결 후 필요한 테이블 가져오기
- charlson
- apsiii
- sapsii
- oasis


In [10]:
# 1. DB 파라미터 설정

# 데이터 저장위치
# output_dir = './data'   # 수정 쌤 디렉토리에서 사용
output_dir = '../../outputs'   # 내 디렉에서 사용


# SSH 연결 설정
ssh_config = {
    'ssh_ip': '210.91.223.248',
    'ssh_port': 35430,
    'ssh_username': "medicalai",
    'ssh_password': "medicalai731!",
    'remote_bind_address': ('localhost', 35432)  # 데이터베이스 호스트 및 포트
}

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_admin',
    'password': 'admin1q2w!@',
    'host': 'localhost',  # SSH 터널을 사용하면, 이 값은 'localhost'가 됩니다
    'port': 35432  # 이 값은 SSH 터널 로컬 포트 바인딩에 의해 대체됩니다
}


# 테이블 쿼리 설정
tables_query = {
    'charlson': "select * from mimiciv_derived.charlson;",
    'apsiii': "select * from mimiciv_derived.apsiii;",
    'sapsii': "select * from mimiciv_derived.sapsii;",
    'oasis': "select * from mimiciv_derived.oasis;"
                }


db.print_config_info(db_config, tables_query)

--------- Database Configuration ---------
Database Name: mimiciv
User: mai_admin
Password: ***********
Host: localhost
Port: 35432

--------- SQL Queries for Required Tables ---------
charlson: select * from mimiciv_derived.charlson;
apsiii: select * from mimiciv_derived.apsiii;
sapsii: select * from mimiciv_derived.sapsii;
oasis: select * from mimiciv_derived.oasis;



In [ ]:
# 2. db 접속, 테이블 추출, 'dataframes' 라는 딕셔너리에 테이블 저장

try:
    conn, tunnel = db.connect_to_database_via_ssh(db_config, ssh_config)
    if conn is not None:
        # 데이터베이스 작업 수행
        dataframes = db.retrieve_data(conn, tables_query)
    else:
        logging.error("Failed to connect to the database.")
except Exception as e:
    logging.error(f"An error occurred: {e}")
finally:
    if conn is not None:
        db.disconnect_database(conn)


# 데이터프레임 변수로 저장
## Severity
charlson = dataframes['charlson']
apsiii = dataframes['apsiii']
sapsii = dataframes['sapsii']
oasis = dataframes['oasis']
